In [ ]:
% run 0-utils.ipynb

In [ ]:
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import MinMaxScaler
from tensorboardX import SummaryWriter

In [ ]:
from gcnn.datasets import load_icebergs
from gcnn.nets import BaselineCNNSimple

In [ ]:
icebergs = load_icebergs('train')

In [ ]:
icebergs.shape

In [ ]:
icebergs.head()

In [ ]:
#icebergs.inc_angle.fillna(icebergs.inc_angle.dropna().median())

In [ ]:
train, test = train_test_split(icebergs, test_size=0.15, stratify=icebergs.is_iceberg)
test.shape

In [ ]:
# rescale between 0-1
scaler_b1 = MinMaxScaler()
scaler_b2 = MinMaxScaler()

In [ ]:
for _, i in train.iterrows():
    scaler_b1.partial_fit(i.band_1.reshape(1, -1))
    scaler_b2.partial_fit(i.band_2.reshape(1, -1))

In [ ]:
train_x = torch.from_numpy(
    np.stack(
        [
            scaler_b1.transform(np.stack(train.band_1)),
            scaler_b2.transform(np.stack(train.band_2)),
        ],
        axis=1
    ).reshape(-1, 2, 75, 75)
).float()

test_x = torch.from_numpy(
    np.stack(
        [
            scaler_b1.transform(np.stack(test.band_1)),
            scaler_b2.transform(np.stack(test.band_2)),
        ],
        axis=1
    ).reshape(-1, 2, 75, 75)
).float()

train_y = torch.from_numpy(train.is_iceberg.values.reshape(-1, 1)).float()
test_y = torch.from_numpy(test.is_iceberg.values.reshape(-1, 1)).float()

In [ ]:
cuda = torch.cuda.is_available()
num_epochs = 25
batch_size = 32
learning_rate = 0.00001
cuda

In [ ]:
train_loader = data_utils.DataLoader(data_utils.TensorDataset(train_x, train_y), batch_size=batch_size, shuffle=True)
test_loader = data_utils.DataLoader(data_utils.TensorDataset(test_x, test_y), batch_size=batch_size, shuffle=True)

In [ ]:
net = BaselineCNNSimple()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
writer = SummaryWriter()
writer.add_graph(net, net(Variable(train_x.narrow(0, 0, 1))))

if cuda:
    net.cuda()
    criterion.cuda()
    
epoch_train_loss = []
epoch_test_loss = []

In [ ]:
def loop(loader, training=False):
    
    running_loss = 0
    targets = []
    predictions = []
    
    for batch_id, (x, y) in enumerate(loader):
        x = Variable(x)
        y = Variable(y)
        
        if cuda:
            x = x.cuda()
            y = y.cuda()
        
        optimizer.zero_grad()
        
        outputs = net(x)
        loss = criterion(outputs, y)

        if training:
            loss.backward()
            optimizer.step()

        running_loss += loss.data.cpu()[0]
        targets.extend(y.data.cpu().numpy())
        predictions.extend(outputs.sigmoid().data.cpu().numpy())
        
    return np.array(targets), np.array(predictions), running_loss

In [ ]:
for e in range(num_epochs):

    train_targets, train_preds, train_loss = loop(train_loader, training=True)
    writer.add_scalar('data/train_loss', train_loss, e)
    writer.add_scalar('data/train_accuracy', accuracy_score(train_targets, train_preds.argmax(axis=1)), e)
      
    test_targets, test_preds, test_loss = loop(test_loader, training=False)
    writer.add_scalar('data/test_loss', test_loss, e)
    writer.add_scalar('data/test_accuracy', accuracy_score(test_targets, test_preds.argmax(axis=1)), e)

    train_loss /= train.shape[0]
    test_loss /= test.shape[0]
    
    epoch_train_loss.append(train_loss)
    epoch_test_loss.append(test_loss)
    
    print('Training loss: {:.4f}'.format(train_loss))
    print('Testing  loss: {:.4f}'.format(test_loss))

In [ ]:
plt.plot(epoch_train_loss, label='train')
plt.plot(epoch_test_loss, label='test')
plt.legend();

In [ ]:
train_targets, train_preds, train_loss = loop(train_loader)
accuracy_score(train_targets, train_preds.round())

In [ ]:
log_loss(train_targets, train_preds)

In [ ]:
test_targets, test_preds, test_loss = loop(test_loader)
accuracy_score(test_targets, test_preds.round())

In [ ]:
log_loss(test_targets, test_preds)